In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### Script for regressions on EU export sanctions

In [2]:
# read conversion table and HS5 level sanction table
HS2022toHS2017 = pd.read_excel('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/HS2022toHS2017.xlsx', sheet_name='Conversions', dtype={'From HS2022': str, 'To HS2017': str}) 
HS5_sanct = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_2/HS5_sanct_short_ext_imp_v2.csv", dtype={'HS2017': str})
HS5_sanct[['Partial_HS5', 'Full_HS5','Transitory_HS5']] = HS5_sanct[['Partial_HS5', 'Full_HS5','Transitory_HS5']].fillna(value=0)
HS5_sanct.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
string_2017 = '201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712'
string_2018 = '201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812'
string_2019 = '201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912'
string_2020 = '202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012'
string_2021 = '202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112'
string_2022 = '202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212'
string_2023 = '202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312'
string_2024 = '202401,202402,202403,202404,202405'

In [5]:
string_dates = [string_2017, string_2018, string_2019, string_2020, string_2021, string_2022, string_2023, string_2024]

In [6]:
import requests
import comtradeapicall
import time
import datetime
subscription_key = '94ee5163870949e1aa6fe904f64e241d' # comtrade api subscription key (from comtradedeveloper.un.org)
directory = '/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Comtrade_v1'  # output directory for downloaded files 
#proxy_url = '<PROXY URL>'  # optional if you need proxy url
# Call preview final data API to a data frame, max to 500 records, no subscription key required
# This example: Australia imports of commodity code 91 in classic mode in May 2022
#typeCode: goods or services
#freqCode: annual or month, eg '202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112'
#period: eg '200001,200002,200003'
#reporterCode: 97 - EU
#partnerCode: 
# 398 - Kzt
# 51 - Armenia
# 417 - Kyrgyzstan
# 792 - Türkiye
# 156 - China
# 756 or 757 - Switzerland
# 643 - Russia

In [7]:
def get_data_exp(params):
    data = comtradeapicall.getFinalData(subscription_key, typeCode='C', freqCode='M', clCode='HS', period=params[0],
                                        reporterCode='97', cmdCode=None, flowCode='M', partnerCode=params[1],
                                        partner2Code=None,
                                        customsCode=None, motCode=None, maxRecords=250000, format_output='JSON',
                                        aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True)
    return data

In [8]:
ECU3 = get_data_exp([string_dates[0], '398,51,417'])

In [9]:
for years in tqdm(string_dates[1:]):
    ECU3_new = get_data_exp([years, '398,51,417'])
    ECU3 = ECU3.append(ECU3_new)


100%|██████████| 7/7 [04:19<00:00, 37.02s/it]


In [52]:
#ECU3.loc[ECU3['refYear'] == 2024, 'cmdCode']
#string_dates[4:-1]

In [26]:
#ECU3.columns
#ECU3['classificationCode'].unique()
# fine
pd.crosstab(index=ECU3['classificationCode'], columns=ECU3['refYear'])

refYear,2017,2018,2019,2020,2021,2022,2023,2024
classificationCode,,,,,,,,
H5,12585,13572,14359,12516,16509,0,0,0
H6,0,0,0,0,0,19714,26129,13650


In [11]:
# keep only needed col-s
#comtrade_cols = ['period', 'partnerCode', 'partnerISO', 'classificationCode', 'cmdCode', 'primaryValue']
comtrade_cols = ['period', 'partnerCode', 'cmdCode', 'primaryValue']
ECU3_short = ECU3[comtrade_cols]
ECU3_short = ECU3_short.loc[ECU3_short['cmdCode'].apply(len) == 6]
ECU3_short.reset_index(inplace=True)
ECU3_short.drop_duplicates(keep='first', inplace=True) # drop duplicates if were downloaded accidentally

In [12]:
#ECU3_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_6/ECU3_short_imp.csv")


In [15]:
# sum ECU3 into one aggregate
# ECU3_short_agg = ECU3_short.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index() # sums over countries
# ECU3_short_agg['partnerCode'] = 'ECU3'
# ECU3_short_agg = ECU3_short_agg[['period', 'partnerCode', 'cmdCode', 'primaryValue']]

In [17]:
#ECU3_short_agg.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/ECU3_short_agg_exp.csv")

In [115]:
RGT = get_data_exp([string_dates[0], '643,268,792'])

In [118]:
for years in tqdm(string_dates[1:]):
    RGT_new = get_data_exp([years, '643,268,792'])
    RGT = RGT.append(RGT_new)



100%|██████████| 4/4 [09:39<00:00, 144.87s/it]


In [185]:
RGT.drop_duplicates(keep='first', inplace=True) # drop duplicates if were downloaded accidentally
RGT_short = RGT[comtrade_cols]
RGT_short = RGT_short.loc[RGT_short['cmdCode'].apply(len) == 6]
RGT_short.reset_index(inplace=True)


In [164]:
#RGT_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/RGT_short_exp.csv")


In [111]:
CW = get_data_exp([string_dates[0], '156,0'])

In [131]:
for years in tqdm(string_dates[1:]):
    CW_new = get_data_exp([years, '156,0'])
    CW = CW.append(CW_new)



100%|██████████| 4/4 [08:12<00:00, 123.08s/it]


In [186]:
CW.drop_duplicates(keep='first', inplace=True)
CW_short = CW[comtrade_cols]
CW_short = CW_short.loc[CW_short['cmdCode'].apply(len) == 6]
CW_short.reset_index(inplace=True)


In [11]:
#CW_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/CW_short_exp.csv")


In [9]:
FSU3 = get_data_exp([string_dates[0], '860,31'])

In [11]:
for years in tqdm(string_dates[1:]):
    FSU3_new = get_data_exp([years, '860,31'])
    FSU3 = FSU3.append(FSU3_new)


100%|██████████| 7/7 [07:28<00:00, 64.13s/it]


In [12]:
# keep only needed col-s
#comtrade_cols = ['period', 'partnerCode', 'partnerISO', 'classificationCode', 'cmdCode', 'primaryValue']
comtrade_cols = ['period', 'partnerCode', 'cmdCode', 'primaryValue']
FSU3_short = FSU3[comtrade_cols]
FSU3_short = FSU3_short.loc[FSU3_short['cmdCode'].apply(len) == 6]
FSU3_short.reset_index(inplace=True)
FSU3_short.drop_duplicates(keep='first', inplace=True) # drop duplicates if were downloaded accidentally


In [14]:
# as wanted
#pd.crosstab(index=FSU3['classificationCode'], columns=FSU3['refYear'])


refYear,2017,2018,2019,2020,2021,2022,2023,2024
classificationCode,,,,,,,,
H5,55365,57350,59647,56402,58154,0,0,0
H6,0,0,0,0,0,62905,66833,27162


In [15]:
#FSU3_short.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/FSU3_short_exp.csv")


In [18]:
# sum ECU3 into one aggregate
# FSU3_short_agg = FSU3_short.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index() # sums over countries
# FSU3_short_agg['partnerCode'] = 'FSU3'
# FSU3_short_agg = FSU3_short_agg[['period', 'partnerCode', 'cmdCode', 'primaryValue']]
# FSU3_short_agg.drop('index', axis=1, inplace=True)

In [24]:
#FSU3_short_agg.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/FSU3_short_agg_exp.csv")


In [13]:
CW_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/CW_short_exp.csv", dtype={'period':str, 'cmdCode': str})
CW_short['cmdCode'] = CW_short['cmdCode'].apply(lambda x: "0"+x if len(x) == 5 else x)

RGT_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/RGT_short_exp.csv", dtype={'period':str, 'cmdCode': str})
ECU3_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/ECU3_short_exp.csv", dtype={'period':str, 'cmdCode': str})
FSU3_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/FSU3_short_exp.csv", dtype={'period':str, 'cmdCode': str})


In [30]:
#CW_short['cmdCode'].apply(type).unique()

In [19]:
database = ECU3_short.append([RGT_short, FSU3_short, CW_short])
database.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
database.drop_duplicates(keep='first', inplace=True)
database['partnerCode'] = database['partnerCode'].apply(str)

In [53]:
#RGT_short.drop(['Unnamed: 0', 'index'], axis=1, inplace=True)
#RGT_short.loc[RGT_short['partnerCode'] == 643]
#database.shape

In [42]:
#database['year'] = database['period'].apply(lambda x: x[:4])
# fine representation
#pd.crosstab(index=database['year'], columns=database['partnerCode'])
#database.drop(['year'], axis=1, inplace=True)

partnerCode,0,31,51,156,268,398,417,643,792,860
year,,,,,,,,,,
2017,63274,22472,18164,50173,25730,29689,11061,49995,50267,15955
2018,63262,22778,18619,50182,26124,28929,11070,49793,49515,17151
2019,63300,23495,19004,49822,26447,29243,11043,49816,49496,18230
2020,63328,21226,17247,47843,23977,27545,9468,49062,48345,17924
2021,63363,21706,18548,48195,25277,28060,10104,49550,49227,18958
2022,65935,22952,23677,48317,27460,32589,16168,41682,50780,21506
2023,65897,24120,25497,48154,28477,34526,23119,32689,51125,23500
2024,27421,9679,10086,20029,11805,13551,9348,12127,21317,9571


In [33]:
#CW_short = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/CW_short_exp.csv", dtype={'period':str, 'cmdCode': str})
#CW_short.loc[CW_short['cmdCode'].apply(len) < 6]

#CW_short['cmdCode'] = CW_short['cmdCode'].apply(lambda x: "0"+x if len(x) == 5 else x)


In [34]:
# database_big = ECU3.append([RGT, CW])
# database_big['classificationCode']
# database_big['classificationCode', 'refYear']
# database_big.columns # 268, 643, 792


# check that HS6 starts inly from 2022
# pd.crosstab(index=database_big['classificationCode'], columns=database_big['refYear'])

# check that all years in å country have approx similar number of codes
# pd.crosstab(index=database_big['partnerCode'], columns=database_big['refYear'])

# checks
# ECU3_short.shape
# ECU3_short.drop_duplicates(keep='first', inplace=True)


In [35]:
world_exp = database.loc[database['partnerCode'] == 0]
world_exp['year'] = world_exp['period'].apply(lambda x: x[:4])


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
world_exp_gr = world_exp.groupby('year')['primaryValue'].sum().reset_index()
print(world_exp_gr)
print(world_exp_gr['primaryValue'].pct_change())

   year  primaryValue
0  2017  2.121816e+12
1  2018  2.311022e+12
2  2019  2.279565e+12
3  2020  2.198331e+12
4  2021  2.576770e+12
5  2022  2.702701e+12
6  2023  2.758787e+12
7  2024  1.153942e+12
0         NaN
1    0.089172
2   -0.013612
3   -0.035636
4    0.172148
5    0.048872
6    0.020752
7   -0.581721
Name: primaryValue, dtype: float64


In [20]:
database['partnerCode'].unique()

array(['51', '398', '417', '268', '643', '792', '31', '860', '156', '0'],
      dtype=object)

In [21]:
# 51, Armenia
# 398, Kazakhstan
# 417, Kyrgyzstan
# 268, Georgia
# 643, Rus
# 792,  Turk
# 31, Azerb
# 860, Uzbekistan
# 156,  China
# 0 World

# compute rest of the world
regres_countr = ['643', '51', '398', '417', '268', '860', '31', '792', '156'] # Rus, ECU, FSU3, Tur, China
regres_countr_data = database.loc[database['partnerCode'].apply(lambda x: x in regres_countr)]
regres_countr_data_agg = regres_countr_data.groupby(['period', 'cmdCode'])['primaryValue'].sum().reset_index()


In [22]:
RestWorld = database.loc[database['partnerCode'] == '0']
RestWorld = pd.merge(RestWorld, regres_countr_data_agg, on=['period', 'cmdCode'], how='left')
RestWorld['primaryValue_y'].fillna(value=0, inplace=True)
RestWorld['primaryValue'] = RestWorld['primaryValue_x'] - RestWorld['primaryValue_y']
RestWorld['partnerCode'] = "RW3"
RestWorld = RestWorld[['period', 'partnerCode', 'cmdCode','primaryValue']]

database = database.append(RestWorld)

In [27]:
# append datasets according to HS5 and HS6 classif
database_2017_2021 = database.loc[database['period'] < '202201']
database_2022_2023 = database.loc[database['period'] >= '202201']

In [49]:
(4*12 + 5)*5400*135


38637000

In [107]:
5*5400*135

3645000

In [28]:
#500 * (12 * 7 + 5)

# convet HS6 to HS5 and group HS6 flows within same month-HS5 group
database_2022_2023_HS5 = pd.merge(database_2022_2023, HS2022toHS2017, left_on='cmdCode', right_on='From HS2022', how='left')
database_2022_2023_HS5 = database_2022_2023_HS5[['period', 'partnerCode', 'primaryValue', 'To HS2017']].groupby(['period', 'partnerCode', 'To HS2017'])['primaryValue'].sum().reset_index()


In [29]:
# rename to have same column names
database_2017_2021.rename(columns={'cmdCode':'cmdCode_HS5'}, inplace=True)
database_2022_2023_HS5.rename(columns={'To HS2017':'cmdCode_HS5'}, inplace=True)

database_HS5 = database_2017_2021.append(database_2022_2023_HS5)


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
database_HS5_sanct = pd.merge(database_HS5, HS5_sanct, left_on='cmdCode_HS5', right_on='HS2017', how='left')
database_HS5_sanct.drop(['HS2017', 'HS2022'], axis=1, inplace=True)
database_HS5_sanct['period'] = database_HS5_sanct['period'].apply(lambda x: pd.to_datetime(x, format='%Y%m')) # convert to timestamp
database_HS5_sanct['Date_HS5'] = database_HS5_sanct['Date_HS5'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d') if x==x else x)
database_HS5_sanct['Date_HS5_similar'] = database_HS5_sanct['Date_HS5_similar'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d') if x==x else x)


In [70]:
# database_HS5_sanct['period'].apply(type).unique()
# database_HS5_sanct['Date_HS5'].apply(type).unique()
#HS5_sanct 
# number of unique similar code
len(database_HS5_sanct.loc[database_HS5_sanct['Date_HS5_similar'].notna(), 'cmdCode_HS5'].unique())

37

Issue: some cmdCode_HS5 were not mathced with HS5_sanct codes, because coversion HS6toHS5 table doesnt contain these HS5 codes (eg because 2 HS5 codes were aggregated into 1 HS6 code). There are 7502 of such rows, so I decide delite them as this should not be large issue.


In [31]:
print(database_HS5_sanct['Sanct_HS5'].isna().sum())
database_HS5_sanct = database_HS5_sanct.loc[database_HS5_sanct['Sanct_HS5'].notna()]


7937


In [63]:
#database_reg['partnerCode'].unique()
#regres_countr.append('RW2') 
#regres_countr + ['RW2']
#regres_countr + ['RW2']
database_HS5_sanct['partnerCode'].unique()

array(['51', '398', '417', '268', '643', '792', '31', '860', '156', '0',
       'RW3'], dtype=object)

In [55]:
regres_countr

['643', '51', '398', '417', '268', '860', '31', '792', '156']

In [54]:
# check on one example that RW is correct
database_HS5_sanct.loc[(database_HS5_sanct['period'] == database_HS5_sanct['period'].iloc[1001]) & (database_HS5_sanct['cmdCode_HS5'] == '300120')]


,period,partnerCode,cmdCode_HS5,primaryValue,Date_HS5,Partial_HS5,Full_HS5,Transitory_HS5,Transitory_firstpatial_HS5,Transitory_firstfull_HS5,Sanct_HS5,Sanct_type,Issue1_HS5,Issue2_HS5,Corr_1:1,Corr_n:1_only,4digit,Date_HS5_similar
60300,2017-03-01,792,300120,1.698915e+03,NaT,0.0,0.0,0.0,NaN,NaN,0.0,Non_sanctioned,0.0,0.0,1.0,0.0,3001.0,NaT
60390,2017-03-01,268,300120,5.132006e+03,NaT,0.0,0.0,0.0,NaN,NaN,0.0,Non_sanctioned,0.0,0.0,1.0,0.0,3001.0,NaT
884640,2017-03-01,0,300120,1.044155e+07,NaT,0.0,0.0,0.0,NaN,NaN,0.0,Non_sanctioned,0.0,0.0,1.0,0.0,3001.0,NaT
884910,2017-03-01,156,300120,7.564980e+02,NaT,0.0,0.0,0.0,NaN,NaN,0.0,Non_sanctioned,0.0,0.0,1.0,0.0,3001.0,NaT
1436769,2017-03-01,RW3,300120,1.043396e+07,NaT,0.0,0.0,0.0,NaN,NaN,0.0,Non_sanctioned,0.0,0.0,1.0,0.0,3001.0,NaT


#### Prepare for regressions

In [57]:
#database_reg = database_HS5_sanct.loc[(database_HS5_sanct['partnerCode'] != 0) & (database_HS5_sanct['partnerCode'] != 156)]
# get delite China and World data
database_reg = database_HS5_sanct.loc[database_HS5_sanct['partnerCode'].apply(lambda x: x in regres_countr + ['RW3'])]


In [64]:
def periodtoint(period):
    if period == period: return period.year*12 + period.month - 24205 # 24240 is number of months before 2020-11-01
    else: return 0

database_reg.sort_values(by=['period'], inplace=True)
#ECU3_sanct_1['Date_HS5'] = ECU3_sanct_1['Date_HS5'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')) # convert to timestamp
database_reg['Date_HS5_int'] = database_reg['Date_HS5'].apply(periodtoint)
database_reg['period_int'] = database_reg['period'].apply(periodtoint)
database_reg['log_primaryValue'] = np.log(database_reg['primaryValue'])


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [65]:
database_reg['Sanct_t'] = (database_reg['period'] > database_reg['Date_HS5']).apply(int)
database_reg['Similar_t'] = (database_reg['period'] > database_reg['Date_HS5_similar']).apply(int)
database_reg['Sanct_t_int'] = (database_reg['period_int'] > database_reg['Date_HS5_int']).apply(int) # becareful, this might be falacious
database_reg['partnerCode'] = database_reg['partnerCode'].apply(str)

# status at t: Partial_t = 1 if partially sanctioned at t, o o/w; Full_t = 1 if fully sanctioned at t, o o/w;
# Thus transitory code has Partial_t = 1 when partially sanctioned and Full_t = 1 when fully
database_reg['Partial_t'] = database_reg['Partial_HS5'] * database_reg['Sanct_t'] + database_reg['Transitory_HS5'] * (((database_reg['period'] > database_reg['Transitory_firstpatial_HS5']) & (database_reg['period'] <= database_reg['Transitory_firstfull_HS5'])).apply(int))
database_reg['Full_t'] = database_reg['Full_HS5'] * database_reg['Sanct_t'] + database_reg['Transitory_HS5'] * ((database_reg['period'] > database_reg['Transitory_firstfull_HS5']).apply(int))


/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [96]:
# database_reg = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_5/database_exp_reg_v3.csv", dtype={'cmdCode_HS5': str})
# database_reg.drop('Unnamed: 0', axis=1, inplace=True)
# check on one example that Similar_t is assigned properly
# database_reg.loc[database_reg['Similar_t']==1, 'cmdCode_HS5'].unique()
# database_reg.loc[(database_reg['cmdCode_HS5']== '392051') & (database_reg['partnerCode']== '643')].iloc[50:80]


In [98]:
# for_similar_4digit_gr = [x[:4] for x in database_reg.loc[database_reg['Sanct_HS5'] == 1, 'cmdCode_HS5'].unique()]
# database_reg['Similar'] = (database_reg['Sanct_type'] == 'Non_sanctioned') & (database_reg['cmdCode_HS5'].apply(lambda x: x[:4] in for_similar_4digit_gr))
# database_reg['Similar'] = database_reg['Similar'].apply(int)

In [103]:
# len(database_reg.loc[database_reg['Similar'] == 1, 'cmdCode_HS5'].unique())
# 8460
# database_reg.shape
# database_reg.loc[database_reg['cmdCode_HS5'].apply(lambda x: x[:4] == '8460'), 'Similar'].sum()
# [x for x in database_reg.loc[database_reg['Similar'] == 1, 'cmdCode_HS5'].unique() if x[:4] == '8460']

In [58]:
# make indicator for "similar" products
# Similar = non_sanctioned which has at least one partially sanctioned 6digit code falling into same 4digit category

# sanct_exp_refin = pd.read_csv('/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_3/sanct_exp_refin_v1.csv', index_col=0, dtype={'CN6':str})


In [59]:
# # only 19 6digit codes are not covered by the 4digit codes (1 code is 2digit that's why 19 but not 20).
# # So the analysis of "similar" goods is not promising here.
# print(sanct_exp_refin['CN6_len'].value_counts())
# giga = sanct_exp_refin.loc[sanct_exp_refin['CN6_len'] == 4, 'CN6'].unique()
# goga = sanct_exp_refin.loc[sanct_exp_refin['CN6_len'] == 2, 'CN6'].unique()
# print(sanct_exp_refin['CN6'].apply(lambda x: x[:4] in giga if len(x) == 6 else False).sum())
# print(sanct_exp_refin['CN6'].apply(lambda x: x[:2] in goga if len(x) == 6 else False).sum())
# print(1656 - 799 - 77) # 854 6digit codes are not covered by 4digit or 2digit sanctioned codes and thus used to find "similar" codes

# for_similar_6digit = sanct_exp_refin.loc[(sanct_exp_refin['CN6'].apply(lambda x: x[:4] not in giga if len(x) == 6 else False) & sanct_exp_refin['CN6'].apply(lambda x: x[:4] not in giga if len(x) == 6 else False)), 'CN6'].unique()


6    1656
4     274
2       3
Name: CN6_len, dtype: int64


In [25]:
# ((database_reg['period'] > database_reg['Transitory_firstpatial_HS5']) & (database_reg['period'] <= database_reg['Transitory_firstfull_HS5'])).sum()

# ((database_reg['period'] >= database_reg['Transitory_firstfull_HS5'])).sum()

# ((database_reg['Transitory_firstpatial_HS5'] <= database_reg['Transitory_firstfull_HS5'])).sum()

#((database_reg['period'] > database_reg['Transitory_firstpatial_HS5']) & (database_reg['period'] <= database_reg['Transitory_firstfull_HS5'])).sum()

# 21146 = 10834 + 10312

10312

In [103]:
# check some transitory codes that Partial_t and Full_t are defined properly
# database_reg.loc[database_reg['Transitory_HS5'] == 1]
# database_reg.loc[(database_reg['cmdCode_HS5'] == '890399') & (database_reg['partnerCode'] == 'ECU3')]
# database_reg.loc[(database_reg['cmdCode_HS5'] == '370500') & (database_reg['partnerCode'] == 'ECU3')]

# check some transitory codes that Partial_t and Full_t are defined properly
# database_reg.loc[database_reg['Full_HS5'] == 1]
# database_reg.loc[(database_reg['cmdCode_HS5'] == '690722') & (database_reg['partnerCode'] == 'ECU3')].iloc[60:70]


In [72]:
# becareful as 0 time_to_treat will code both, treatment period and non-sanct products => always use with sanctioned indicator to distinguish
database_reg['time_to_treat'] = database_reg['period_int'] - database_reg['Date_HS5_int']
database_reg.loc[database_reg['Sanct_HS5'] == 0, 'time_to_treat'] = 0

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [105]:
(database_reg['Sanct_t'] != database_reg['Sanct_t_int']).sum()

982272

In [73]:
#database_reg.loc[(database_reg['Sanct_t'] != database_reg['Sanct_t_int'])]
#database_reg
#dummies = 

def time_to_treat(x):
    if x < -12: return -13
    elif x > 12: return 13
    else: return x
    
database_reg['time_to_treat_ES1'] = database_reg['time_to_treat'].apply(time_to_treat)    

/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [76]:
database_reg = pd.concat([database_reg, pd.get_dummies(database_reg['partnerCode']).rename(columns={'643':'Rus', '51':'Arm', '398':'Kaz', 
                                                            '417':'Kyr', '31':'Aze', '268':'Geo',
                                                            '860':'Uzb', '792':'Tur', 'RW3':'RW3', '156':'Chi'})], axis=1)

In [75]:
database_reg['partnerCode'].unique()

array(['51', '792', '643', '31', 'RW3', '268', '156', '860', '417', '398'],
      dtype=object)

In [77]:
# dummies for ECU3 and FSU3
database_reg['ECU3'] = database_reg['Kaz'] + database_reg['Kyr'] + database_reg['Arm']
database_reg['FSU3'] = database_reg['Geo'] + database_reg['Aze'] + database_reg['Uzb']
database_reg['ECU3'] = database_reg['ECU3'].apply(lambda x: int(x>0))
database_reg['FSU3'] = database_reg['FSU3'].apply(lambda x: int(x>0))


In [78]:
database_reg

,period,partnerCode,cmdCode_HS5,primaryValue,Date_HS5,Partial_HS5,Full_HS5,Transitory_HS5,Transitory_firstpatial_HS5,Transitory_firstfull_HS5,...,Aze,Kaz,Kyr,Arm,Rus,Tur,Uzb,RW3,ECU3,FSU3
0,2017-01-01,51,081310,1.231224e+03,NaT,0.0,0.0,0.0,NaN,NaN,...,0,0,0,1,0,0,0,0,1,0
44187,2017-01-01,792,411200,2.517099e+05,NaT,0.0,0.0,0.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
44188,2017-01-01,792,411310,2.332533e+04,NaT,0.0,0.0,0.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
44189,2017-01-01,792,411330,1.021470e+05,NaT,0.0,0.0,0.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
44190,2017-01-01,792,411390,1.662471e+05,NaT,0.0,0.0,0.0,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555994,2024-05-01,31,843691,5.847144e+04,NaT,0.0,0.0,0.0,NaN,NaN,...,1,0,0,0,0,0,0,0,0,1
2555993,2024-05-01,31,843610,2.952062e+04,NaT,0.0,0.0,0.0,NaN,NaN,...,1,0,0,0,0,0,0,0,0,1
2555992,2024-05-01,31,843510,1.595885e+03,NaT,0.0,0.0,0.0,NaN,NaN,...,1,0,0,0,0,0,0,0,0,1
2555990,2024-05-01,31,843360,1.806974e+06,NaT,0.0,0.0,0.0,NaN,NaN,...,1,0,0,0,0,0,0,0,0,1


In [79]:

#database_reg.to_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_6/database_imp_reg_v2.csv")


In [3]:
database_reg = pd.read_csv("/Users/omega_macbookpro/Desktop/International_Trade/Dissertation_MRes/Comtrade/Sanctions Docs/EU/Data/Output_Script_6/database_imp_reg_v2.csv")



/Users/omega_macbookpro/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
database_reg.head()

,Unnamed: 0,period,partnerCode,cmdCode_HS5,primaryValue,Date_HS5,Partial_HS5,Full_HS5,Transitory_HS5,Transitory_firstpatial_HS5,...,Aze,Kaz,Kyr,Arm,Rus,Tur,Uzb,RW3,ECU3,FSU3
0,0,2017-01-01,51,81310,1231.224,NaN,0.0,0.0,0.0,NaN,...,0,0,0,1,0,0,0,0,1,0
1,44187,2017-01-01,792,411200,251709.949,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
2,44188,2017-01-01,792,411310,23325.326,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
3,44189,2017-01-01,792,411330,102147.013,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0
4,44190,2017-01-01,792,411390,166247.082,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,1,0,0,0,0


In [5]:
database_reg.columns # 1624 - full, 121 - partial, 34

Index(['Unnamed: 0', 'period', 'partnerCode', 'cmdCode_HS5', 'primaryValue',
       'Date_HS5', 'Partial_HS5', 'Full_HS5', 'Transitory_HS5',
       'Transitory_firstpatial_HS5', 'Transitory_firstfull_HS5', 'Sanct_HS5',
       'Sanct_type', 'Issue1_HS5', 'Issue2_HS5', 'Corr_1:1', 'Corr_n:1_only',
       '4digit', 'Date_HS5_similar', 'Date_HS5_int', 'period_int',
       'log_primaryValue', 'Sanct_t', 'Similar_t', 'Sanct_t_int', 'Partial_t',
       'Full_t', 'time_to_treat', 'time_to_treat_ES1', 'Chi', 'Geo', 'Aze',
       'Kaz', 'Kyr', 'Arm', 'Rus', 'Tur', 'Uzb', 'RW3', 'ECU3', 'FSU3'],
      dtype='object')

In [18]:
len([x for x in database_reg.loc[database_reg['Partial_t'] == 1, 'cmdCode_HS5'].unique() if x not in database_reg.loc[database_reg['Full_t'] == 1, 'cmdCode_HS5'].unique()])





63

In [16]:
290514 in database_reg.loc[database_reg['Full_t'] == 1, 'cmdCode_HS5'].unique()

True

In [108]:
database_reg['time_to_treat_ES1'].unique()


array([-13,   0, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,
        -1,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13])

In [113]:
11/17, 400/600, 1552.18/ 2659.21, (2933.44 - 1552.18)/(4328.42 - 2659.21)

(0.6470588235294118,
 0.6666666666666666,
 0.5836996702028046,
 0.8274932453076604)

In [118]:
#15/24, 8/12
5/17, 4/11

(0.29411764705882354, 0.36363636363636365)

In [2]:
#database_reg['Similar_t'].isna().sum()

In [1]:
#database_reg.loc[database_reg['Sanct_t'] != database_reg['Sanct_t_int']]
# Sanct_HS5
#database_reg[database_reg.columns[:15]]  #.loc[database_reg['Sanct_t'] != database_reg['Sanct_t_int']]

In [105]:
import numpy as np
np.sort(database_reg['time_to_treat'].unique())


array([-85, -84, -83, -82, -81, -80, -79, -78, -77, -76, -75, -74, -73,
       -72, -71, -70, -69, -68, -67, -66, -65, -64, -63, -62, -61, -60,
       -59, -58, -57, -56, -55, -54, -53, -52, -51, -50, -49, -48, -47,
       -46, -45, -44, -43, -42, -41, -40, -39, -38, -37, -36, -35, -34,
       -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21,
       -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,
        -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,
         6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27])

In [ ]:
# for transitory add make sanct_t for when period >= Transitory_firstfull_HS5

### Desc Stat for codes


In [3]:
HS5_sanct['Similar'] = HS5_sanct['Date_HS5_similar'].notna().apply(int)

In [4]:
HS5_sanct['Similar'] = HS5_sanct['Date_HS5_similar'].notna().apply(int)

HS5_sanct_desc = HS5_sanct[['Sanct_HS5', 'Full_HS5', 'Partial_HS5',
       'Transitory_HS5', 'Similar', 'Issue2_HS5',
       'Issue1_HS5']]

In [5]:
print((HS5_sanct['Corr_1:1'] == 1).sum())
print((HS5_sanct['Corr_n:1_only'] == 1).sum()) 
print((HS5_sanct['Issue2_HS5'] == 1).sum())
print((HS5_sanct['Issue1_HS5'] == 1).sum())




4097
65
1225
87


In [6]:
sums = pd.DataFrame(HS5_sanct_desc.sum()).T
sums.rename(index={0:'Sum'}, inplace=True)
HS5_sanct_desc = HS5_sanct_desc.describe().append(sums)

HS5_sanct_desc.rename(columns={'Sanct_HS5':'Sanctioned', 
                               'Full_HS5': 'Full', 'Partial_HS5': 'Partial', 'Transitory_HS5': 'PartialToFull', 
                               'Issue2_HS5': 'Flag 1', 'Issue1_HS5': 'Flag 2'}, inplace=True)

HS5_sanct_desc_short = HS5_sanct_desc.loc[['Sum', 'mean', 'count']]

In [7]:
HS5_sanct_desc_short.head()

,Sanctioned,Full,Partial,PartialToFull,Similar,Flag 1,Flag 2
Sum,1692.000000,1570.000000,64.000000,58.000000,37.0000,1225.00000,87.000000
mean,0.315554,0.292801,0.011936,0.010817,0.0069,0.22846,0.016225
count,5362.000000,5362.000000,5362.000000,5362.000000,5362.0000,5362.00000,5362.000000


In [8]:
print(round(HS5_sanct_desc_short, 3).to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &  Sanctioned &      Full &   Partial &  PartialToFull &   Similar &    Flag 1 &    Flag 2 \\
\midrule
Sum   &    1692.000 &  1570.000 &    64.000 &         58.000 &    37.000 &  1225.000 &    87.000 \\
mean  &       0.316 &     0.293 &     0.012 &          0.011 &     0.007 &     0.228 &     0.016 \\
count &    5362.000 &  5362.000 &  5362.000 &       5362.000 &  5362.000 &  5362.000 &  5362.000 \\
\bottomrule
\end{tabular}

